## Install and Import Dependencies

In [1]:
# import Libraries
import mediapipe as mp
import cv2
import glob
import numpy as np
import csv
import os
from mat4py import loadmat
import scipy.io as sio
import math
from os import listdir
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline, FeatureUnion
import pickle 
from math import cos ,sin
from imutils import face_utils
import imutils
from sklearn.svm import SVR

In [2]:
# Drawing Helper
mp_drawing = mp.solutions.drawing_utils
# Mediapipe Solutions
mp_holistic = mp.solutions.holistic

##  Collect Data & Export to CSV

In [3]:
paths = []
# get the path or directory
folder_dir = "E:\ITI\AFLW2000"
for images in glob.iglob(f'{folder_dir}/*'):
    if (images.endswith(".jpg")):
        paths.append(images) 

In [4]:
print(paths[2])

E:\ITI\AFLW2000\image00006.jpg


In [5]:
len(paths)

2000

In [6]:
paths_mat=[]
# get the path or directory
folder_dir = "E:\ITI\AFLW2000"
for images in glob.iglob(f'{folder_dir}/*'):
    if (images.endswith(".mat")):
        paths_mat.append(images)

In [7]:
print(paths_mat[2])

E:\ITI\AFLW2000\image00006.mat


In [8]:
len(paths_mat)

2000

In [9]:
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
mp_holistic = mp.solutions.holistic

In [10]:
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

In [11]:
x_stack = np.array([np.linspace(1,468, num= 468)])
y_stack = np.array([np.linspace(1,468, num= 468)])


# ypr_stack = np.array([np.linspace(1,3, num= 3)])
pitch_array1 = []
yaw_array1 = []
roll_array1 = []

x_array_init = []
y_array_init = []
all_ypr = []
for indx , img in enumerate(paths) :
    x_array_face = []
    y_array_face = []
    image = cv2.imread(img)
    image = imutils.resize(image, width=450)
  # cv2_imshow(image)
    result = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    face = result.multi_face_landmarks

    try:
        for landm in face[0].landmark:
      
            x = landm.x
            y = landm.y
            # note: the x and y values are Normalized from 0 to 1 and we need to denormalize them
            relative_x = int(x * image.shape [1])
            relative_y = int(y * image.shape [0])
            x_array_face.append(relative_x)
            y_array_face.append(relative_y)
    
        x_array_face = np.array(x_array_face)
        x_stack = np.vstack([x_stack,x_array_face.T])

        y_array_face = np.array(y_array_face)
        y_stack = np.vstack([y_stack,y_array_face.T])

        # x_array_init.append(x_array_face)
        # y_array_init.append(y_array_face)

        # full_x_array = np.array(x_array_init)
        # full_y_array = np.array(y_array_init)

        mat_file = sio.loadmat(paths_mat[indx])
        # extracting the labels 3 angels
        pose_para = mat_file["Pose_Para"][0][:3]
        pitch = pose_para[0]
        yaw = pose_para[1]
        roll = pose_para[2] 
        pitch_array1.append(pitch)
        yaw_array1.append(yaw)
        roll_array1.append(roll)

        pitch_array = np.array(pitch_array1)

        yaw_array = np.array(yaw_array1)

        roll_array = np.array(roll_array1)

        # ypr_stack = np.vstack([ypr_stack, all_ypr_array.T])


    except:
        pass

In [12]:
pitch_array[0]

-0.3992307782173157

In [13]:
# # Uncomment for the first time only
x_stack = np.delete(x_stack, 0, 0)
x_stack.shape

(1853, 468)

In [14]:
# # Uncomment for the first time only
y_stack = np.delete(y_stack,0,0)
y_stack.shape

(1853, 468)

In [15]:
print(pitch_array.shape)
print(yaw_array.shape)
print(roll_array.shape)

(1853,)
(1853,)
(1853,)


In [16]:
# center 99
# Top of Head = 10
# Bottom of Head = 171

# centering the data arround the 99th point 
x_center = x_stack - x_stack[:,99].reshape(-1,1)
y_center = y_stack - y_stack[:,99].reshape(-1,1)

# normalizing the data to be in the same scale by dividing over the distance between point 10 and point 171
x_171 = x_stack[:,171]
x_10 = x_stack[:,10]
y_171 = y_stack[:,171]
y_10 = y_stack[:,10]

# computing the distance
distance = np.linalg.norm(np.array((x_10,y_10)) - np.array((x_171,y_171)),axis = 0).reshape(-1,1)
x_norm = x_center / distance
y_norm = y_center / distance

print(x_norm.shape)
print(y_norm.shape)

# concatinating the X and Y points to form the compelete features
features = np.hstack([x_norm,y_norm])

print(features.shape)

(1853, 468)
(1853, 468)
(1853, 936)


## Read Data 

In [17]:
df = pd.DataFrame(features)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
0,0.042774,0.051328,0.051328,0.029942,0.051328,0.059883,0.076993,-0.192482,0.085547,0.089825,...,0.008555,-0.004277,0.000000,0.012832,0.017109,-0.290861,-0.278029,-0.260920,-0.316526,-0.333635
1,0.000000,-0.070821,0.019315,-0.012876,-0.070821,-0.038629,0.064382,0.064382,0.096573,0.090135,...,-0.019315,-0.025753,0.012876,-0.012876,0.000000,-0.283282,-0.270406,-0.263967,-0.302597,-0.321911
2,0.061069,0.071247,0.061069,0.020356,0.071247,0.071247,0.050891,-0.264632,0.040713,0.040713,...,-0.010178,-0.020356,-0.020356,0.000000,0.000000,-0.274810,-0.264632,-0.254454,-0.305344,-0.315523
3,0.081445,0.005090,0.050903,-0.055994,-0.015271,-0.025452,-0.035632,-0.244336,-0.071265,-0.091626,...,-0.025452,-0.045813,-0.061084,-0.015271,-0.015271,-0.361413,-0.341052,-0.320691,-0.463220,-0.488671
4,0.067940,0.031705,0.045293,-0.018117,0.022647,0.013588,-0.004529,-0.240055,-0.018117,-0.027176,...,-0.031705,-0.049823,-0.031705,-0.027176,-0.018117,-0.289878,-0.276290,-0.267231,-0.348759,-0.357818


In [18]:
data = df.copy()
data['yaw'] = pd.DataFrame(yaw_array)
data['pitch'] = pd.DataFrame(pitch_array)
data['roll'] = pd.DataFrame(roll_array)
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,yaw,pitch,roll
0,0.042774,0.051328,0.051328,0.029942,0.051328,0.059883,0.076993,-0.192482,0.085547,0.089825,...,0.012832,0.017109,-0.290861,-0.278029,-0.260920,-0.316526,-0.333635,0.018227,-0.399231,0.085676
1,0.000000,-0.070821,0.019315,-0.012876,-0.070821,-0.038629,0.064382,0.064382,0.096573,0.090135,...,-0.012876,0.000000,-0.283282,-0.270406,-0.263967,-0.302597,-0.321911,1.189533,0.470065,0.300959
2,0.061069,0.071247,0.061069,0.020356,0.071247,0.071247,0.050891,-0.264632,0.040713,0.040713,...,0.000000,0.000000,-0.274810,-0.264632,-0.254454,-0.305344,-0.315523,0.881137,-0.184650,-0.236852
3,0.081445,0.005090,0.050903,-0.055994,-0.015271,-0.025452,-0.035632,-0.244336,-0.071265,-0.091626,...,-0.015271,-0.015271,-0.361413,-0.341052,-0.320691,-0.463220,-0.488671,0.299208,-0.175379,-0.373374
4,0.067940,0.031705,0.045293,-0.018117,0.022647,0.013588,-0.004529,-0.240055,-0.018117,-0.027176,...,-0.027176,-0.018117,-0.289878,-0.276290,-0.267231,-0.348759,-0.357818,0.011965,-0.026812,-0.220662


In [19]:
data.to_csv("landmarks.csv")

In [20]:
# features
x= np.array(df)
print(x.shape)
# pitch label
pa=data['pitch']
# yaw label
ya=data['yaw']
# roll label
ra=data['roll']
#labels
Angles=data[['pitch','yaw','roll']]

(1853, 936)


In [21]:
x

array([[ 0.04277373,  0.05132847,  0.05132847, ..., -0.26091974,
        -0.31652559, -0.33363508],
       [ 0.        , -0.07082051,  0.01931468, ..., -0.26396734,
        -0.30259671, -0.32191139],
       [ 0.0610689 ,  0.07124705,  0.0610689 , ..., -0.25445375,
        -0.3053445 , -0.31552265],
       ...,
       [ 0.0302791 ,  0.07065123,  0.05046516, ..., -0.2170002 ,
        -0.20186065, -0.2170002 ],
       [ 0.04213974,  0.05899563,  0.05056768, ..., -0.26126636,
        -0.3034061 , -0.31604802],
       [ 0.05792269,  0.0534671 ,  0.04901151, ..., -0.25842433,
        -0.28961347, -0.30743584]])

In [22]:
pa.shape

(1853,)

In [23]:
ya.shape

(1853,)

In [24]:
ra.shape

(1853,)

In [25]:
Angles

,pitch,yaw,roll
0,-0.399231,0.018227,0.085676
1,0.470065,1.189533,0.300959
2,-0.184650,0.881137,-0.236852
3,-0.175379,0.299208,-0.373374
4,-0.026812,0.011965,-0.220662
...,...,...,...
1848,-0.306358,-0.283822,0.038554
1849,-0.367547,-0.429723,0.122791
1850,-0.156035,0.567114,-0.108536
1851,-0.197102,-0.070430,0.105118


##  Prepare Data

### 1.PITCH

In [26]:
X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(x, pa, test_size=0.2,shuffle=True, random_state=1234)

X_val_p, X_test_p, y_val_p, y_test_p = train_test_split(X_val_p, y_val_p, test_size=0.5,shuffle=True, random_state=1234)

In [27]:
X_train_p.shape

(1482, 936)

In [28]:
X_val_p.shape

(185, 936)

In [29]:
y_test_p.shape

(186,)

### 2. YAW

In [30]:
X_train_y, X_val_y, y_train_y, y_val_y = train_test_split(x, ya, test_size=0.2, shuffle=True,random_state=1234)

X_val_y, X_test_y, y_val_y, y_test_y = train_test_split(X_val_y, y_val_y, test_size=0.5, shuffle=True,random_state=1234)

In [31]:
X_train_y.shape

(1482, 936)

In [32]:
X_val_y.shape

(185, 936)

In [33]:
y_test_y.shape

(186,)

### 3. Roll

In [34]:
X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(x, ra, test_size=0.2,shuffle=True, random_state=1234)

X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_val_r, y_val_r, test_size=0.5,shuffle=True, random_state=1234)

In [35]:
X_train_r.shape

(1482, 936)

In [36]:
X_val_r.shape

(185, 936)

In [37]:
y_test_r.shape

(186,)

## Traning Models and Prediction

### 1. PITCH

In [38]:
pipelines = {
        'rf_pca_pa':make_pipeline(StandardScaler(),PCA(n_components=100), SVR())
    #     ,'lr_pca_pa':make_pipeline(PCA(n_components=0.99),LinearRegression()),
    #     'DT_pa'    :make_pipeline(DecisionTreeRegressor()),
    #     'DT_pa_pa'    :make_pipeline(PCA(n_components=0.99),DecisionTreeRegressor()),


}

In [39]:
pitch_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_p, y_train_p)
    pitch_fit_models[algo] = model

In [40]:
pitch_fit_models

{'rf_pca_pa': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('pca', PCA(n_components=100)), ('svr', SVR())])}

In [41]:
Train_pred_pitch     = pitch_fit_models['rf_pca_pa'].predict(X_train_p)
Validation_pred_pitch = pitch_fit_models['rf_pca_pa'].predict(X_val_p)
pitch_Angle=pitch_fit_models['rf_pca_pa'].predict(X_test_p)

In [42]:
################################## Error Evaluation (Mean Square Error)#####################################################
# Traning Data MSE
print("Train MSE: "       , mean_squared_error(y_train_p,Train_pred_pitch))
# Validation Data MSE
print("Validation MSE: "  , mean_squared_error(y_val_p,Validation_pred_pitch))
# test Data MSE
print("Test MSE: "  , mean_squared_error(y_test_p,pitch_Angle))

Train MSE:  0.1810325682636633
Validation MSE:  0.04083820030556943
Test MSE:  0.8785186120867401


In [43]:
e_points_p = (np.sqrt(mean_squared_error(y_test_p,pitch_Angle)) /3.14) * 180 
e_points_p

53.730185559310875

In [44]:
################################## Model Evaluation (r2_score)#####################################################

print("Train r2_score: "            , r2_score(y_train_p,Train_pred_pitch))
print("Validation r2_score: "       , r2_score(y_val_p,Validation_pred_pitch))
print("Test r2_score: "             , r2_score(y_test_p , pitch_Angle))

Train r2_score:  0.27287198423840986
Validation r2_score:  0.602035182175459
Test r2_score:  0.040608827503253786


In [45]:
################################## Model Evaluation (score)#####################################################
print("Train score "                   , pitch_fit_models['rf_pca_pa'].score(X_train_p,y_train_p))
print("Validation Score "              , pitch_fit_models['rf_pca_pa'].score(X_val_p,y_val_p))
print("test score: "                   , pitch_fit_models['rf_pca_pa'].score(X_test_p , y_test_p))

Train score  0.27287198423840986
Validation Score  0.602035182175459
test score:  0.040608827503253786


In [46]:
with open('pitch_model.pkl', 'wb') as f:
    pickle.dump(pitch_fit_models ['rf_pca_pa'], f)

### 2. YAW

In [47]:
pipelines = {
        'rf_pca_ya':make_pipeline(StandardScaler(),PCA(n_components=100), SVR())
    #     ,'lr_pca_pa':make_pipeline(PCA(n_components=0.99),LinearRegression()),
    #     'DT_pa'    :make_pipeline(DecisionTreeRegressor()),
    #     'DT_pa_pa'    :make_pipeline(PCA(n_components=0.99),DecisionTreeRegressor()),


}

In [48]:
yaw_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_y, y_train_y)
    yaw_fit_models[algo] = model

In [49]:
yaw_fit_models

{'rf_pca_ya': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('pca', PCA(n_components=100)), ('svr', SVR())])}

In [50]:
Train_pred_yaw     = yaw_fit_models['rf_pca_ya'].predict(X_train_y)
Validation_pred_yaw = yaw_fit_models['rf_pca_ya'].predict(X_val_y)
yaw_Angle=yaw_fit_models['rf_pca_ya'].predict(X_test_y)

In [51]:
################################## Error Evaluation (Mean Square Error)#####################################################
# Traning Data MSE
print("Train MSE: "       , mean_squared_error(y_train_y,Train_pred_yaw))
# Validation Data MSE
print("Validation MSE: "  , mean_squared_error(y_val_y,Validation_pred_yaw))
# test Data MSE
print("Test MSE: "  , mean_squared_error(y_test_y,yaw_Angle))

Train MSE:  0.021691505489683835
Validation MSE:  0.018479158076174198
Test MSE:  0.193232081712843


In [52]:
e_points_y = (np.sqrt(mean_squared_error(y_test_y,yaw_Angle)) /3.14) * 180 
e_points_y

25.198951655569907

In [53]:
################################## Model Evaluation (r2_score)#####################################################

print("Train r2_score: "            , r2_score(y_train_y,Train_pred_yaw))
print("Validation r2_score: "       , r2_score(y_val_y,Validation_pred_yaw))
print("Test r2_score: "             , r2_score(y_test_y , yaw_Angle))

Train r2_score:  0.9321847681843457
Validation r2_score:  0.9378303870751212
Test r2_score:  0.6176682626021923


In [54]:
################################## Model Evaluation (score)#####################################################
print("Train score "                   , yaw_fit_models['rf_pca_ya'].score(X_train_y,y_train_y))
print("Validation Score "              , yaw_fit_models['rf_pca_ya'].score(X_val_y,y_val_y))
print("test score: "                   , yaw_fit_models['rf_pca_ya'].score(X_test_y , y_test_y))

Train score  0.9321847681843457
Validation Score  0.9378303870751212
test score:  0.6176682626021923


In [55]:
with open('yaw_model.pkl', 'wb') as f:
    pickle.dump(yaw_fit_models ['rf_pca_ya'], f)

### 3. ROLL

In [56]:
pipelines = {
        'rf_pca_ra':make_pipeline(StandardScaler(),PCA(n_components=100), SVR())
    #     ,'lr_pca_pa':make_pipeline(PCA(n_components=0.99),LinearRegression()),
    #     'DT_pa'    :make_pipeline(DecisionTreeRegressor()),
    #     'DT_pa_pa'    :make_pipeline(PCA(n_components=0.99),DecisionTreeRegressor()),


}

In [57]:
roll_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_r, y_train_r)
    roll_fit_models[algo] = model

In [58]:
roll_fit_models

{'rf_pca_ra': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('pca', PCA(n_components=100)), ('svr', SVR())])}

In [59]:
Train_pred_roll     = roll_fit_models['rf_pca_ra'].predict(X_train_r)
Validation_pred_roll = roll_fit_models['rf_pca_ra'].predict(X_val_r)
roll_Angle=roll_fit_models['rf_pca_ra'].predict(X_test_r)

### Model evaluation

In [60]:
################################## Error Evaluation (Mean Square Error)#####################################################
# Traning Data MSE
print("Train MSE: "       , mean_squared_error(y_train_r,Train_pred_roll))
# Validation Data MSE
print("Validation MSE: "  , mean_squared_error(y_val_r,Validation_pred_roll))
# test Data MSE
print("Test MSE: "  , mean_squared_error(y_test_r,roll_Angle))

Train MSE:  0.18071368027822418
Validation MSE:  0.04329673595501177
Test MSE:  1.3261148848120707


In [61]:
e_points_r = (np.sqrt(mean_squared_error(y_test_r,roll_Angle)) /3.14) * 180 
e_points_r

66.01360222038939

In [62]:
################################## Model Evaluation (r2_score)#####################################################

print("Train r2_score: "            , r2_score(y_train_r,Train_pred_roll))
print("Validation r2_score: "       , r2_score(y_val_r,Validation_pred_roll))
print("Test r2_score: "             , r2_score(y_test_r , roll_Angle))

Train r2_score:  0.4006751171381191
Validation r2_score:  0.6410431100825953
Test r2_score:  0.20425009666441285


In [63]:
################################## Model Evaluation (score)#####################################################
print("Train score "                   , roll_fit_models['rf_pca_ra'].score(X_train_r,y_train_r))
print("Validation Score "              , roll_fit_models['rf_pca_ra'].score(X_val_r,y_val_r))
print("test score: "                   , roll_fit_models['rf_pca_ra'].score(X_test_r , y_test_r))

Train score  0.4006751171381191
Validation Score  0.6410431100825953
test score:  0.20425009666441285


In [64]:
with open('roll_model.pkl', 'wb') as f:
    pickle.dump(roll_fit_models ['rf_pca_ra'], f)

## Load Models 

In [72]:
with open('yaw_model.pkl', 'rb') as f:
    yaw_model = pickle.load(f)

In [73]:
with open('pitch_model.pkl', 'rb') as f:
    pitch_model = pickle.load(f)

In [74]:
with open('roll_model.pkl', 'rb') as f:
    roll_model = pickle.load(f)

## Drawing 3 Axis


In [75]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [78]:
faceModule = mp.solutions.face_mesh

In [79]:
#  Create a VideoCapture object and read from input file
def Create_TestVideo(pitch_model,yaw_model,roll_model,path):
  cap = cv2.VideoCapture(path)
  width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
      print("Error opening video file")
  #a list to store the frames of the video  
  video_array = []
  # Read until video is completed
  while(cap.isOpened()):   
    # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
        # loading image 
        with faceModule.FaceMesh(static_image_mode=True) as faces:
          # processing the face to extract the landmark points (468 point) for each x,y,z
          results = faces.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
          if results.multi_face_landmarks != None: 
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                # initializing two lists to store the points for every face.
                x_array = []
                y_array = []
                # looping over the 468 points of x and y
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
                    shape = frame.shape 
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])
                    #Xs of face
                    x_array.append(relative_x)
                    #Ys of face
                    y_array.append(relative_y)

                # converting the x anx y to numpy arrays
                x_array = np.array(x_array)
                y_array = np.array(y_array)

                # centering the points arround the 99th point 
                x_center = x_array - x_array[99]
                y_center = y_array - y_array[99]

                # computing the distance
                distance = np.linalg.norm(np.array((x_array[171],y_array[171])) - np.array((x_array[10],y_array[10])),axis = 0)
                x_norm = x_center / distance
                y_norm = y_center / distance

                # concatinating the x and y points to predict the labels (pitch,yaw,roll)
                features = np.hstack([x_norm,y_norm]).reshape(1,-1)

                # predicting the 3 angels to draw the axis on the image
                pred_pitch = pitch_model.predict(features)
                pred_yaw = yaw_model.predict(features)
                pred_roll = roll_model.predict(features)
                draw_axis(frame,pred_pitch,pred_yaw,pred_roll,x_array[99],y_array[99])
                # appending the result frames to the video_array list
                video_array.append(frame)
      else:
        break
  cap.release()  
  # Closes all the frames
  cv2.destroyAllWindows()
  print("Number of Detected Frames = ",len(video_array))
  # converting the frames to video
  out = cv2.VideoWriter('Test_video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))
  for i in range(len(video_array)):
    out.write(video_array[i])
  out.release()


In [80]:
path = "C:/Users/Omar/OneDrive/Pictures/Camera Roll/Project.mp4" 
Create_TestVideo(pitch_model,yaw_model,roll_model,path)

Number of Detected Frames =  412
